# Notebook 1 - Education and Salary - a first overview in California.

This notebook aims to create a single dataframe collecting data from the _Census.Gov_ and the _Bureau of Economic Analysis_ page with information on:
1. Education Levels - Namely the percentage of Over25 that hold a Bachelor's degree
2. Education Institution Presence - from the Firms API we analysed the "educational services" category (number 61) that showed us per each county how many educ. centres were present (this includes kindergarten, schools of any level and university)
3. Salary - The total income of the county [Source: US Bureau of Economic Analysis]

**PLEASE NOTE** In this notebook we started testing the API and the possibility of a single dataframe. For this notebook only, the data will be taken from the state of California, while in the next steps we extend our research to the remaining states.


We start by importing the necessary libraries, retrieving the data from the API and analysing only the state of California for the sake of notebook size, we will proceed to a further analysis in a later step of our project

In [1]:
#Required imports for the project
import requests # for api requests
import pandas as pd #tabular data

from bs4 import BeautifulSoup

API Query.

In [ ]:
# !! REMINDER TO TAKE API KEY OUT OF CODE BEFORE SUBMITTING !!
api_key = ""

In [ ]:
#creating a dataframe function
def json_to_dataframe(response):
    return pd.DataFrame(response.json()[1:], columns=response.json()[0])

In [ ]:
#Creating a function to remove the leading zeros from a column
def remove_leading_zeros(df, column):
    df[column] = df[column].str.lstrip('0')
    return df

In [ ]:
#Setting up the API Query parameters for the educational services industry
params1 = {"NAICS2017" : 61}

In [ ]:
#Requesting the json file from the census website using the api key
url = "https://api.census.gov/data/2017/ecnbasic?get=NAICS2017_LABEL,NAICS2017,GEO_ID,FIRM&for=county:*&key={}".format(api_key)
response = requests.request("GET", url, params=params1)

In [ ]:
response.text[0:77]

'[["NAICS2017_LABEL","NAICS2017","GEO_ID","FIRM","NAICS2017","state","county"]'

In [ ]:
#Converting ecn data frame into pandas data frame
educational_services = json_to_dataframe(response)
df = pd.DataFrame(data = educational_services)
df['FIRM'] = df['FIRM'].astype(int)
df['state'] = df['state'].astype("string")
df_california1 = df[df['state'] == '06']

In [ ]:
# Top 5 counties with largest number of educational services
df_c_sort = df_california1\
.sort_values(by=['FIRM'], ascending=False)\
.head(10)
print('Top 10 counties with the most amount of educational services in California')
df_c_sort

Top 10 counties with the most amount of educational services in California


,NAICS2017_LABEL,NAICS2017,GEO_ID,FIRM,NAICS2017,state,county
1439,Educational services,61,0500000US06037,2787,61,06,037
309,Educational services,61,0500000US06059,1187,61,06,059
324,Educational services,61,0500000US06073,1010,61,06,073
296,Educational services,61,0500000US06085,822,61,06,085
1419,Educational services,61,0500000US06001,693,61,06,001
213,Educational services,61,0500000US06075,470,61,06,075
209,Educational services,61,0500000US06067,334,61,06,067
313,Educational services,61,0500000US06081,323,61,06,081
294,Educational services,61,0500000US06065,313,61,06,065
195,Educational services,61,0500000US06013,295,61,06,013


In [ ]:
#Number of counties in the enc dataset for California
df_california1['county'].count()

48

In [ ]:
#Setting the params
params2 = {"state" : "06"}

In [ ]:
#Getting the ACS data
#Requesting the json file from the census website using the api key 
url = "https://api.census.gov/data/2017/acs/acs1/profile?get=DP02_0064PE,DP02_0088PE,DP02_0123PE&for=county&key={}".format(api_key)
response2 = requests.request("GET", url)

In [ ]:
#Seeing what the columns for the data are
response2.text[0:61]

'[["DP02_0064PE","DP02_0088PE","DP02_0123PE","state","county"]'

## Meaning of variables
- **DP02_0064PE** = Percent!!EDUCATIONAL ATTAINMENT!!Population 25 years and over!!Bachelor's degree

In [ ]:
pop_chars = json_to_dataframe(response2)
df = pd.DataFrame(data = pop_chars)
df_california2 = df[df['state'] == '06']

In [ ]:
#Number of counties in the acs dataset for California
df_california2['county'].count()

40

In [ ]:
#Merging the two datasets
merged_census = pd.merge(df_california1, df_california2, on='county')
#drop state_y column and rename state_x to state
merged_census = merged_census.drop(columns=['state_y', 'NAICS2017'])
merged_census = merged_census.rename(columns={'state_x': 'state', 'FIRM':'Number of Educational Institutions','DP02_0064PE':'Percent of Population with a Bachelor\'s Degree'})
#Convert the columns to the correct data types
merged_census['Percent of Population with a Bachelor\'s Degree'] = merged_census['Percent of Population with a Bachelor\'s Degree'].astype(float)
merged_census['county'] = merged_census['county'].astype(int)
# The last two columns are currently unneeded but will be later used for contextual analysis.
#Drop the last two columns
merged_census = merged_census.drop(columns=['DP02_0088PE', 'DP02_0123PE'])

We finally create a merged_census file containing the Number of Educational Institutions and the Percent of Population with a Bachelor's Degree.

In [ ]:
merged_census.head()

,NAICS2017_LABEL,GEO_ID,Number of Educational Institutions,state,county,Percent of Population with a Bachelor's Degree
0,Educational services,0500000US06047,13,06,47,8.7
1,Educational services,0500000US06033,8,06,33,9.5
2,Educational services,0500000US06013,295,06,13,26.7
3,Educational services,0500000US06083,137,06,83,21.2
4,Educational services,0500000US06115,0,06,115,13.9


For easier reference, we attach the County Name through the State and County codes. 

In [ ]:
merged_census

,NAICS2017_LABEL,GEO_ID,Number of Educational Institutions,state,county,Percent of Population with a Bachelor's Degree
0,Educational services,0500000US06047,13,06,47,8.7
1,Educational services,0500000US06033,8,06,33,9.5
2,Educational services,0500000US06013,295,06,13,26.7
3,Educational services,0500000US06083,137,06,83,21.2
4,Educational services,0500000US06115,0,06,115,13.9
5,Educational services,0500000US06099,71,06,99,12.0
6,Educational services,0500000US06097,161,06,97,22.6
7,Educational services,0500000US06025,5,06,25,10.4
8,Educational services,0500000US06067,334,06,67,20.0
9,Educational services,0500000US06017,37,06,17,22.7


To create the graph, we have to remove some of the variables.

In [ ]:
#Create a copy of merged_census called education_data
education_data = merged_census.copy()
#Clean up GEO_ID column by removing "0500000US" from the string
education_data['GEO_ID'] = education_data['GEO_ID'].str.replace('0500000US', '')
education_data.head(5)

,NAICS2017_LABEL,GEO_ID,Number of Educational Institutions,state,county,Percent of Population with a Bachelor's Degree
0,Educational services,06047,13,06,47,8.7
1,Educational services,06033,8,06,33,9.5
2,Educational services,06013,295,06,13,26.7
3,Educational services,06083,137,06,83,21.2
4,Educational services,06115,0,06,115,13.9


## Getting the Code on National Income

In [ ]:
import pandas as pd
#import excel file and convert to dataframe
us_gdp = pd.read_excel("../data/us_gdp_2009-18.xlsx")
# if GeoFIPS is under 10000, add a 0 in front of it
us_gdp['Clean GeoFIPS'] = us_gdp['Clean GeoFIPS'].apply(lambda x: '{0:0>5}'.format(x))
#convert GeoFIPS to string
us_gdp['Clean GeoFIPS'] = us_gdp['Clean GeoFIPS'].astype('string')
#keep only Clean GEOFIPS and GDP per Capita 2017
us_gdp = us_gdp[['Clean GeoFIPS', 'GDP per Capita 2017']]
#rename Clean GeoFIPS to GEO_ID
us_gdp = us_gdp.rename(columns={'Clean GeoFIPS': 'GEO_ID'})
us_gdp.head(5)

,GEO_ID,GDP per Capita 2017
0,01001,25.839051
1,01003,27.172905
2,01005,28.457883
3,01007,15.671346
4,01009,14.114381


In [ ]:
#Merge the gdp dataframe with the education_data dataframe
merged_gdp = pd.merge(education_data, us_gdp, on='GEO_ID')
merged_gdp.head(5)

,NAICS2017_LABEL,GEO_ID,Number of Educational Institutions,state,county,Percent of Population with a Bachelor's Degree,GDP per Capita 2017
0,Educational services,06047,13,06,47,8.7,35.545039
1,Educational services,06033,8,06,33,9.5,32.864295
2,Educational services,06013,295,06,13,26.7,67.186104
3,Educational services,06083,137,06,83,21.2,60.634992
4,Educational services,06115,0,06,115,13.9,39.997401


In [ ]:
# Create a scatterplot of the data with on the x-axis the percent of population with a Bachelor's Degree and on the y-axis the GDP per capita, the size of the bubbles is the number of educational institutions using plotnine
from plotnine import *
#Add a title to the plot
(ggplot(merged_gdp, aes(x='Percent of Population with a Bachelor\'s Degree', y='GDP per Capita 2017', size='Number of Educational Institutions')) + geom_point() + ggtitle("GDP vs Education in California") + theme(axis_text_x=element_text(angle=90)))


ModuleNotFoundError: No module named 'plotnine'

In [ ]:
# Calculate the coefficient of correlation between the two variables
merged_gdp['Percent of Population with a Bachelor\'s Degree'].corr(merged_gdp['GDP per Capita 2017'])

0.7228024336203954

A correlation is evident here.